# Part V — APIs with Django  
## 24. Django REST Framework (DRF) Core (Serializers, Views, ViewSets, Routers, Auth, Pagination, Tests)

This chapter rebuilds your “plain Django JSON API” into a **proper, scalable API**
using **Django REST Framework** (DRF) the way it’s done in real teams.

You’ll learn DRF’s core building blocks deeply:

- **Serializers** (validation + JSON representation)
- **Requests/Responses** (`request.data`, `Response`, renderers)
- **APIView / GenericAPIView** vs **ViewSets**
- **Routers** (automatic URL wiring)
- **Authentication/Permissions** (correct API security)
- **Pagination + filtering**
- **API tests** (APIClient)

---

## 24.0 Compatibility and Version Pinning (Important in 2026)

As of DRF **3.16.1**, official support includes Django **4.2, 5.0, 5.1, 5.2** and
Python **>= 3.9** (per DRF release info). If you’re currently on Django **6.0+**,
you may need to pin Django down to a supported version or verify DRF compatibility.

### 24.0.1 Check versions
```bash
python -c "import django; print('Django', django.get_version())"
python -c "import sys; print(sys.version)"
```

### 24.0.2 Recommended “industry standard” baseline for this workbook
- Django **5.2.x** (LTS series)
- DRF **3.16.1**

Pin DRF (and optionally django-filter) in `requirements.txt`:

```text
djangorestframework==3.16.1
django-filter==24.3
```

Install:
```bash
python -m pip install -r requirements.txt
```

---

## 24.1 What DRF Adds on Top of Django

Without DRF, you manually:
- parse JSON from `request.body`
- validate types and required fields
- build consistent error responses
- handle content negotiation
- implement pagination metadata
- implement auth/permissions patterns consistently

DRF provides a standard stack:
- serializers (validation + transformation)
- `request.data` (parsers)
- `Response` (renderers)
- permission classes + authentication classes
- pagination/filtering helpers
- APIClient test utilities
- browsable API UI (very useful for dev teams)

---

## 24.2 Install and Enable DRF

### 24.2.1 Add `rest_framework` to `INSTALLED_APPS`
In `config/settings.py` (or `base.py`):

```python
INSTALLED_APPS = [
    # Django apps...
    "django.contrib.admin",
    "django.contrib.auth",
    "django.contrib.contenttypes",
    "django.contrib.sessions",
    "django.contrib.messages",
    "django.contrib.staticfiles",
    # Your apps...
    "pages",
    "articles",
    "orgs",
    "tasks",
    # Third-party apps...
    "rest_framework",
]
```

### 24.2.2 Add DRF login URLs (browsable API)
In `config/urls.py`:

```python
from django.contrib import admin
from django.urls import include, path

urlpatterns = [
    path("admin/", admin.site.urls),
    path("accounts/", include("django.contrib.auth.urls")),
    path("api-auth/", include("rest_framework.urls")),
    path("articles/", include("articles.urls")),
    path("orgs/", include("orgs.urls")),
    path("", include("pages.urls")),
]
```

#### Why `api-auth/` exists
It enables login/logout in the browsable API.
This is dev-only convenience but extremely productive.

---

## 24.3 DRF Global Settings (Permissions, Auth, Pagination Defaults)

Add a `REST_FRAMEWORK` dict in settings:

```python
REST_FRAMEWORK = {
    "DEFAULT_AUTHENTICATION_CLASSES": [
        "rest_framework.authentication.SessionAuthentication",
    ],
    "DEFAULT_PERMISSION_CLASSES": [
        "rest_framework.permissions.IsAuthenticatedOrReadOnly",
    ],
    "DEFAULT_PAGINATION_CLASS": (
        "rest_framework.pagination.PageNumberPagination"
    ),
    "PAGE_SIZE": 20,
}
```

### 24.3.1 What this means
- Session authentication:
  - works great for same-site API + browsable API
  - requires CSRF for unsafe methods (POST/PATCH/DELETE)
- Default permission:
  - anonymous can read (GET)
  - must log in for writes
- Page number pagination:
  - list endpoints return `count/next/previous/results`

Later (advanced) you might change to token auth and different pagination styles.

---

## 24.4 Your First DRF Endpoint: Articles API (Read-Only)

We will build:

- `GET /api/v1/articles/`
- `GET /api/v1/articles/<slug>/`

### Why read-only first
It lets you learn serializers + viewsets + routers without mixing create/update
permissions and CSRF complexity immediately.

---

## 24.5 Serializers (The Core DRF Concept)

A serializer does two jobs:

1. **Serialization**: Python object → JSON-friendly dict
2. **Deserialization/Validation**: input data → validated Python data

### 24.5.1 Create `articles/api/serializers.py`
Create folders:

```text
articles/
  api/
    __init__.py
    serializers.py
    views.py
    urls.py
```

Now create `articles/api/serializers.py`:

```python
from rest_framework import serializers

from articles.models import Article, Tag


class TagSerializer(serializers.ModelSerializer):
    class Meta:
        model = Tag
        fields = ["name", "slug"]


class ArticleListSerializer(serializers.ModelSerializer):
    author_username = serializers.CharField(
        source="author.username",
        read_only=True,
    )
    tags = serializers.SlugRelatedField(
        many=True,
        read_only=True,
        slug_field="slug",
    )

    class Meta:
        model = Article
        fields = [
            "id",
            "slug",
            "title",
            "status",
            "published_at",
            "author_username",
            "tags",
        ]


class ArticleDetailSerializer(serializers.ModelSerializer):
    author = serializers.SerializerMethodField()
    tags = TagSerializer(many=True, read_only=True)

    class Meta:
        model = Article
        fields = [
            "id",
            "slug",
            "title",
            "body",
            "status",
            "published_at",
            "author",
            "tags",
        ]

    def get_author(self, obj):
        return {
            "id": obj.author_id,
            "username": obj.author.username,
        }
```

### 24.5.2 Why we used different serializers for list vs detail
Industry standard:
- list endpoints return **summaries**
- detail endpoints return **full content**

This reduces payload size and speeds up list pages for mobile clients.

### 24.5.3 Why `SlugRelatedField` for tags in list
In lists, you often only need tag identifiers:

```json
"tags": ["django", "routing"]
```

In detail, you might want full tag objects:

```json
"tags": [{"name": "Django", "slug": "django"}]
```

This is a practical tradeoff between bandwidth and richness.

---

## 24.6 Views: ViewSets (Best Default for CRUD-ish Resources)

Create `articles/api/views.py`:

```python
from rest_framework import permissions, viewsets

from articles.models import Article
from articles.selectors import article_qs_for_public
from articles.api.serializers import (
    ArticleDetailSerializer,
    ArticleListSerializer,
)


class ArticleViewSet(viewsets.ReadOnlyModelViewSet):
    permission_classes = [permissions.AllowAny]
    lookup_field = "slug"

    def get_queryset(self):
        return article_qs_for_public()

    def get_serializer_class(self):
        if self.action == "retrieve":
            return ArticleDetailSerializer
        return ArticleListSerializer
```

### 24.6.1 Explain each line that matters

- `ReadOnlyModelViewSet` gives you:
  - `.list()` for GET collection
  - `.retrieve()` for GET single resource
- `lookup_field = "slug"` makes detail URLs use slug instead of id.
- `get_queryset()` centralizes:
  - published-only visibility
  - `select_related/prefetch_related` via your selector
- `get_serializer_class()` picks list vs detail serializer based on action.

This is a clean, scalable pattern.

---

## 24.7 Routers and URLs (How DRF Auto-Wires Endpoints)

Create `articles/api/urls.py`:

```python
from django.urls import include, path
from rest_framework.routers import DefaultRouter

from articles.api.views import ArticleViewSet

router = DefaultRouter()
router.register("articles", ArticleViewSet, basename="article")

urlpatterns = [
    path("", include(router.urls)),
]
```

Now include it under `/api/v1/` at project level.

In `config/urls.py`:

```python
from django.contrib import admin
from django.urls import include, path

urlpatterns = [
    path("admin/", admin.site.urls),
    path("accounts/", include("django.contrib.auth.urls")),
    path("api-auth/", include("rest_framework.urls")),
    path("api/v1/", include("articles.api.urls")),
    path("articles/", include("articles.urls")),
    path("orgs/", include("orgs.urls")),
    path("", include("pages.urls")),
]
```

### 24.7.1 Why we use `/api/v1/`
This is path-based versioning:
- explicit
- easy to route and document
- industry common

---

## 24.8 Try the API (Browsable UI + curl)

### 24.8.1 Browsable API
Run server:
```bash
python manage.py runserver
```

Visit:
- `http://127.0.0.1:8000/api/v1/articles/`

You should see:
- browsable API list
- pagination fields (`count`, `next`, `previous`, `results`) once pagination is enabled

### 24.8.2 curl
```bash
curl -i http://127.0.0.1:8000/api/v1/articles/
curl -i http://127.0.0.1:8000/api/v1/articles/hello-django/
```

---

## 24.9 Filtering and Searching (Core Concepts; Full Filtering Later)

DRF supports filtering in multiple ways. The simplest “core DRF” way is manual
query param parsing in `get_queryset()`. (Later we’ll use `django-filter`.)

Add simple search to articles list.

Update `ArticleViewSet.get_queryset()`:

```python
from django.db.models import Q

def get_queryset(self):
    qs = article_qs_for_public()
    q = (self.request.query_params.get("q") or "").strip()
    tag = (self.request.query_params.get("tag") or "").strip()

    if tag:
        qs = qs.filter(tags__slug=tag)

    if q:
        qs = qs.filter(Q(title__icontains=q) | Q(body__icontains=q))

    return qs.distinct()
```

### 24.9.1 Why `request.query_params`
In DRF:
- `request.query_params` is the API-friendly alias for query parameters.
- Similar to `request.GET`, but DRF uses it consistently across request types.

---

## 24.10 Write Endpoints: Tasks API (Org‑Scoped + Role Authorization)

Now we build an API that demonstrates real-world complexity:

- org scoping in URL
- membership required
- role-based permissions
- object-level edit permissions

We’ll implement:

- `GET /api/v1/orgs/<org_slug>/tasks/` (list)
- `POST /api/v1/orgs/<org_slug>/tasks/` (create)
- `GET /api/v1/orgs/<org_slug>/tasks/<id>/` (retrieve)
- `PATCH /api/v1/orgs/<org_slug>/tasks/<id>/` (update)
- `DELETE /api/v1/orgs/<org_slug>/tasks/<id>/` (delete)

### 24.10.1 Serializer: `tasks/api/serializers.py`

Create:

```text
tasks/
  api/
    __init__.py
    serializers.py
    permissions.py
    views.py
    urls.py
```

`tasks/api/serializers.py`:

```python
from rest_framework import serializers

from tasks.models import Task


class TaskSerializer(serializers.ModelSerializer):
    assigned_to_username = serializers.CharField(
        source="assigned_to.username",
        read_only=True,
    )
    created_by_username = serializers.CharField(
        source="created_by.username",
        read_only=True,
    )
    updated_by_username = serializers.CharField(
        source="updated_by.username",
        read_only=True,
    )

    class Meta:
        model = Task
        fields = [
            "id",
            "title",
            "description",
            "status",
            "priority",
            "due_date",
            "assigned_to",
            "assigned_to_username",
            "created_by_username",
            "updated_by_username",
            "created_at",
            "updated_at",
        ]
        read_only_fields = [
            "created_at",
            "updated_at",
            "created_by_username",
            "updated_by_username",
            "assigned_to_username",
        ]
```

### 24.10.2 Why we expose both `assigned_to` and `assigned_to_username`
- `assigned_to` (id) is useful for clients to set relationships.
- `assigned_to_username` is convenient for display without extra user fetch.

This is a pragmatic API design choice.

---

## 24.11 Permissions in DRF (Custom Permission Class)

We already have task permission logic in `tasks.permissions` from Chapter 22:

- `can_edit_task(membership, user, task)`
- `can_delete_task(membership)`

We’ll wrap that in a DRF permission class.

`tasks/api/permissions.py`:

```python
from rest_framework.permissions import BasePermission

from orgs.services import get_membership, get_org_for_user_or_404
from tasks.permissions import can_delete_task, can_edit_task
from tasks.models import Task


class IsOrgMember(BasePermission):
    """
    Ensures the user is authenticated and belongs to the org in the URL.

    We use org_slug from the URL and org membership checks from orgs.services.
    """

    def has_permission(self, request, view):
        if not request.user.is_authenticated:
            return False

        org_slug = view.kwargs.get("org_slug")
        if not org_slug:
            return False

        # This raises 404 if not a member.
        get_org_for_user_or_404(user=request.user, org_slug=org_slug)
        return True


class TaskObjectPermissions(BasePermission):
    """
    Object-level permissions for editing/deleting tasks inside an org.
    """

    def has_object_permission(self, request, view, obj: Task):
        org_slug = view.kwargs.get("org_slug")
        org = get_org_for_user_or_404(user=request.user, org_slug=org_slug)
        membership = get_membership(user=request.user, organization=org)

        if request.method in {"GET", "HEAD", "OPTIONS"}:
            return True

        if request.method in {"PUT", "PATCH"}:
            return can_edit_task(
                membership=membership,
                user=request.user,
                task=obj,
            )

        if request.method == "DELETE":
            return can_delete_task(membership=membership)

        return False
```

### 24.11.1 Why we split permission checks
- `IsOrgMember` handles **global access** to the org-scoped endpoint.
- `TaskObjectPermissions` handles **object-level permissions**.

This is a clean, testable pattern.

---

## 24.12 ViewSet for Tasks (Scoped QuerySets + Service Layer Writes)

`tasks/api/views.py`:

```python
from rest_framework import viewsets
from rest_framework.permissions import IsAuthenticated

from orgs.services import get_org_for_user_or_404
from tasks.api.permissions import IsOrgMember, TaskObjectPermissions
from tasks.api.serializers import TaskSerializer
from tasks.models import Task
from tasks.services import create_task_from_form, update_task_from_form
from tasks.forms import TaskForm
from tasks.selectors import task_qs_for_org


class TaskViewSet(viewsets.ModelViewSet):
    serializer_class = TaskSerializer
    permission_classes = [
        IsAuthenticated,
        IsOrgMember,
        TaskObjectPermissions,
    ]

    def get_queryset(self):
        org = get_org_for_user_or_404(
            user=self.request.user,
            org_slug=self.kwargs["org_slug"],
        )
        return task_qs_for_org(organization=org).order_by("-created_at")

    def perform_create(self, serializer):
        """
        We do NOT trust client to set organization/created_by/updated_by.
        Organization comes from URL + membership check.
        """
        org = get_org_for_user_or_404(
            user=self.request.user,
            org_slug=self.kwargs["org_slug"],
        )

        # Use TaskForm + service layer for consistent audit/event logic.
        form = TaskForm(data=serializer.initial_data)
        form.is_valid(raise_exception=True)

        result = create_task_from_form(
            form=form,
            organization=org,
            actor=self.request.user,
        )
        serializer.instance = result.task

    def perform_update(self, serializer):
        # Use TaskForm + service layer again for event logging.
        instance = self.get_object()
        form = TaskForm(data=serializer.initial_data, instance=instance)
        form.is_valid(raise_exception=True)

        result = update_task_from_form(form=form, actor=self.request.user)
        serializer.instance = result.task
```

### 24.12.1 Why this looks “unusual”
DRF typically uses `serializer.save()` directly. But your project already has a
service layer that:
- sets audit fields
- logs TaskEvent rows
- ensures consistent workflow

In real codebases, it’s common to integrate DRF with service layers to avoid
duplicating business rules.

### 24.12.2 Alternative (simpler) approach
You *can* do:
- `serializer.save(...)` with `created_by`, `updated_by`, `organization`
But then you must manually create TaskEvent etc. The service layer keeps it consistent.

---

## 24.13 Task API URLs (Nested Under Org)

`tasks/api/urls.py`:

```python
from django.urls import include, path
from rest_framework.routers import DefaultRouter

from tasks.api.views import TaskViewSet

router = DefaultRouter()
router.register("tasks", TaskViewSet, basename="org-tasks")

urlpatterns = [
    path("", include(router.urls)),
]
```

Now include it under `orgs` in the API version routing.

Create `orgs/api/urls.py`:

```python
from django.urls import include, path

urlpatterns = [
    path("orgs/<slug:org_slug>/", include("tasks.api.urls")),
]
```

Include in `config/urls.py` under `/api/v1/`:

```python
path("api/v1/", include("articles.api.urls")),
path("api/v1/", include("orgs.api.urls")),
```

You now have:

- `/api/v1/articles/`
- `/api/v1/orgs/acme/tasks/`

---

## 24.14 CSRF in DRF (SessionAuthentication Gotcha)

Because we used `SessionAuthentication`, unsafe methods (POST/PATCH/DELETE) require
CSRF token in browser contexts.

### 24.14.1 Browsable API handles CSRF for you (when logged in)
If you use the browsable API UI, it includes CSRF token automatically.

### 24.14.2 curl needs CSRF only if you use session auth
For API clients (mobile apps, third-party), you typically won’t use session auth.
You’ll use token auth later, which avoids CSRF.

For now:
- use browsable API for write testing
- or implement token auth in the next API security chapter

---

## 24.15 API Tests (DRF’s APIClient)

DRF provides `APIClient` and `APITestCase` which make JSON testing easier.

### 24.15.1 Articles API tests
Create `articles/api/tests.py`:

```python
from django.urls import reverse
from django.utils import timezone
from rest_framework.test import APITestCase

from articles.models import Article
from tests.factories import ArticleFactory


class ArticleApiTests(APITestCase):
    def test_list_returns_paginated_shape(self):
        ArticleFactory(
            status=Article.Status.PUBLISHED,
            published_at=timezone.now(),
        )

        url = reverse("article-list")
        response = self.client.get(url)

        self.assertEqual(response.status_code, 200)
        payload = response.json()

        self.assertIn("count", payload)
        self.assertIn("results", payload)
        self.assertIsInstance(payload["results"], list)

    def test_detail_by_slug(self):
        a = ArticleFactory(
            slug="hello-django",
            status=Article.Status.PUBLISHED,
            published_at=timezone.now(),
        )

        url = reverse("article-detail", kwargs={"slug": a.slug})
        response = self.client.get(url)

        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.json()["slug"], "hello-django")
```

#### Note about reverse names
DRF router names are:
- `<basename>-list`
- `<basename>-detail`

We registered basename `"article"` in router. That yields:
- `article-list`
- `article-detail`

(If your basename differs, adjust accordingly.)

### 24.15.2 Tasks API tests (membership + permissions)
Create `tasks/api/tests.py`:

```python
from django.urls import reverse
from rest_framework.test import APITestCase

from orgs.models import Membership
from tests.factories import MembershipFactory, OrganizationFactory, TaskFactory, UserFactory


class TaskApiTests(APITestCase):
    def setUp(self):
        self.org = OrganizationFactory(slug="acme")

        self.admin = UserFactory(username="admin")
        self.member = UserFactory(username="member")
        self.outsider = UserFactory(username="outsider")

        MembershipFactory(
            organization=self.org,
            user=self.admin,
            role=Membership.Role.ADMIN,
        )
        MembershipFactory(
            organization=self.org,
            user=self.member,
            role=Membership.Role.MEMBER,
        )

        self.task = TaskFactory(
            organization=self.org,
            created_by=self.admin,
            updated_by=self.admin,
            assigned_to=self.member,
        )

    def test_outsider_cannot_list_tasks(self):
        self.client.login(username="outsider", password="pass12345")

        url = reverse("org-tasks-list", kwargs={"org_slug": "acme"})
        response = self.client.get(url)

        # Our org membership function uses 404 for non-members.
        self.assertEqual(response.status_code, 404)

    def test_member_can_list_tasks(self):
        self.client.login(username="member", password="pass12345")

        url = reverse("org-tasks-list", kwargs={"org_slug": "acme"})
        response = self.client.get(url)

        self.assertEqual(response.status_code, 200)
        self.assertIn("results", response.json())

    def test_member_cannot_delete_task(self):
        self.client.login(username="member", password="pass12345")

        url = reverse(
            "org-tasks-detail",
            kwargs={"org_slug": "acme", "pk": self.task.id},
        )
        response = self.client.delete(url)

        self.assertEqual(response.status_code, 403)

    def test_admin_can_delete_task(self):
        self.client.login(username="admin", password="pass12345")

        url = reverse(
            "org-tasks-detail",
            kwargs={"org_slug": "acme", "pk": self.task.id},
        )
        response = self.client.delete(url)

        self.assertIn(response.status_code, {204, 200})
```

#### Why `pk` in URL kwargs
DefaultRouter uses `pk` for detail route unless you change lookup field. We used
`TaskViewSet` default lookup, so `pk` is correct.

---

## 24.16 What You Should Now Understand (DRF “Core Map”)

### Serializers
- define output structure
- validate input
- handle nested relations and computed fields

### ViewSets + Routers
- fast path to standard CRUD endpoints
- consistent URL naming
- smaller URLconf

### Permissions + Authentication
- enforce access rules centrally
- integrate with your existing policy/service layer

### Pagination
- stable response shapes for collections
- avoids huge payloads by default

---

## 24.17 Exercises (Do These Before Proceeding)

1. Add tag filtering to articles API:
   - `GET /api/v1/articles/?tag=django`
   - write an API test that confirms only matching articles are returned.

2. Add ordering to articles API with whitelist:
   - allow `ordering=published_at` and `ordering=-published_at`
   - reject unknown ordering with 400.

3. Add a “my tasks” filter:
   - `GET /api/v1/orgs/acme/tasks/?assigned_to=me`
   - implement in `get_queryset()` or selector filter
   - test it.

4. Add a write endpoint for comments (optional):
   - `POST /api/v1/articles/<slug>/comments/`
   - validate body length
   - return 201 on success with created comment representation.

---

Next chapter: **Part V — 25. DRF Advanced Patterns**  
We’ll cover: nested writes, object-level permissions the “DRF way”, performance
(select_related/prefetch in serializers), throttling, file uploads in APIs, API
testing patterns, and consistent error shaping—plus production-grade filtering via
`django-filter`.